# **Code begins**
using augmented images to train data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import glob
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import Augmentor

**augmented model**

In [ ]:
all_train_label = np.load("D:/NUS_ACADEMY/00computer_intelligence/CAs/CA1/with_augmentation/all_train_label_aug.npy")

In [ ]:
class_label = np.unique(all_train_label)
class_label

In [ ]:
label_to_id = {v:k for k,v in enumerate(class_label) }
id_to_label = {v:k for k,v in label_to_id.items() }

id_to_label

In [ ]:
all_train_label_id = np.array([label_to_id[i] for i in all_train_label])
all_train_label_id

In [ ]:
# import augmented dataset
X_train = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\train_fruit_img.npy")
Y_train = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\train_label_id.npy")
X_validate = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\validate_fruit_img.npy")
Y_validate = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\validate_label_id.npy")

X_test = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\test_fruit_img.npy")
Y_test = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\test_label_id.npy")

mean = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\meanfile.npy")
std = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\Augmented dataset\\stdfile.npy")
mean = mean.reshape(1,1,1,3)
std = std.reshape(1,1,1,3)

## Approach 2
X_train = X_train.astype(np.float32, copy=False)
X_validate = X_validate.astype(np.float32,copy = False)
X_test = X_test.astype(np.float32,copy = False)

X_train -= mean
X_train /= std
X_validate -= mean
X_validate /= std
X_test -= mean
X_test /= std

## flatten data
X_flat_train = X_train.reshape(X_train.shape[0],32*32*3)
X_flat_validate = X_validate.reshape(X_validate.shape[0],32*32*3)
X_flat_test = X_test.reshape(X_test.shape[0],32*32*3)

print('Original Sizes:', X_train.shape, X_validate.shape, Y_train.shape, Y_validate.shape,X_test.shape, Y_test.shape)
print('Flattened:', X_flat_train.shape, X_flat_validate.shape, X_flat_test.shape)

In [ ]:
clf = MLPClassifier(solver='sgd', activation='relu', alpha=1e-5,
                    hidden_layer_sizes=(32*3,32),random_state=1, 
                    verbose = True, learning_rate = 'constant',
                    learning_rate_init=0.01, warm_start=True)

clf.fit(X_flat_train,Y_train)

In [ ]:
joblib.dump(clf,"D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_Bestmodel.sav")

In [ ]:
print("clf.n_iter: "+ str(clf.n_iter_))

In [ ]:
print("train set score: %f" % clf.score(X_flat_train,Y_train))
print("validate set score: %f" % clf.score(X_flat_validate,Y_validate))
print("test set score: %f" % clf.score(X_flat_test,Y_test))

In [ ]:
aug_model = joblib.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_Bestmodel.sav")
print("augmented model train set score: %f" % aug_model.score(X_flat_train,Y_train))
print("augmented model validate set score: %f" % aug_model.score(X_flat_validate,Y_validate))
print("augmented model test set score: %f" % aug_model.score(X_flat_test,Y_test))

In [ ]:
import matplotlib
fig = plt.figure()
plt.plot(clf.loss_curve_)
plt.ylabel("loss per epoch")
plt.xlabel("epochs")
plt.title("MLFF-BP training loss normalized by z-score")
plt.show()
plt.close(fig)

In [ ]:
# N_iter = 30
N_iter = clf.n_iter_
clf1 = MLPClassifier(max_iter=1,solver='sgd', activation='relu', alpha=1e-5,
                    hidden_layer_sizes=(32*3,32),random_state=1,learning_rate = 'constant', 
                    verbose = True, learning_rate_init=0.01, 
                    warm_start=True)
train_score = []
validate_score = []
Best_score = 0
Best_epoch = 0
Bestmodel = "D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\test.sav"

for iter in range(0,N_iter):
  clf1.fit(X_flat_train,Y_train)
  train_score_new = clf1.score(X_flat_train,Y_train)
  validate_score_new = clf1.score(X_flat_validate,Y_validate)
  if validate_score_new > Best_score:
    Best_score = validate_score_new
    Best_epoch = iter
    joblib.dump(clf1,Bestmodel)    
  train_score.append(train_score_new)
  validate_score.append(validate_score_new)
  
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_Best_epoch.npy",Best_epoch)
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_train_score.npy",train_score)
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\ugmented_Best_score.npy",Best_score)
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_validate_score.npy",validate_score)
np.savetxt('D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_train_score.txt', train_score, delimiter = ',')
np.savetxt('D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_validate_score.txt', validate_score, delimiter = ',')


In [ ]:
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_Best_epoch.npy",Best_epoch)
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_train_score.npy",train_score)
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\ugmented_Best_score.npy",Best_score)
np.save("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_validate_score.npy",validate_score)

In [ ]:
#Plot history for accuracy and loss
def plot_model(model_details, train_acc_score, val_acc_score):
    # Create sub-plots
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    
    # Summarize history for accuracy
    axs[0].plot(range(1,len(train_acc_score)+1),train_acc_score)
    axs[0].plot(range(1,len(val_acc_score)+1),val_acc_score)
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(train_acc_score)+1),len(train_acc_score)/10)
    axs[0].legend(['train', 'validate'], loc='best')
    
    # Summarize history for loss
    axs[1].plot(range(1,len(model_details.loss_curve_)+1),model_details.loss_curve_)
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_details.loss_curve_)+1),len(model_details.loss_curve_)/10)
    axs[1].legend(['train', 'validate'], loc='best')

    # Show the plot
    plt.show()
    

# Plot confusion matrix of prediction value
def plot_confusion_matrix(cm, classes):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, '',
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

In [ ]:
plot_model(clf1, train_score, validate_score) 

In [ ]:
print("train set score: %f" % clf1.score(X_flat_train,Y_train))
print("validate set score: %f" % clf1.score(X_flat_validate,Y_validate))
print("test set score: %f" % clf1.score(X_flat_test,Y_test))

In [ ]:
## import shi yuan's non-augmented data
## test using non-augmented mean and std
X_train_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\train_fruit_img.npy")
Y_train_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\train_label_id.npy")
X_validate_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\validate_fruit_img.npy")
Y_validate_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\validate_label_id.npy")

X_test_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\test_fruit_img.npy")
Y_test_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\test_label_id.npy")

mean_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\meanfile.npy")
std_org = np.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\jupyter_shared\\No image augmentation dataset\\stdfile.npy")
mean_org = mean.reshape(1,1,1,3)
std_org = std.reshape(1,1,1,3)

## Approach 2
X_train_org = X_train.astype(np.float32, copy=False)
X_validate_org = X_validate.astype(np.float32,copy = False)
X_test_org = X_test.astype(np.float32,copy = False)

X_train_org -= mean
X_train_org /= std
X_validate_org -= mean
X_validate_org /= std
X_test_org -= mean
X_test_org /= std

## flatten data
X_flat_train_org = X_train_org.reshape(X_train_org.shape[0],32*32*3)
X_flat_validate_org = X_validate_org.reshape(X_validate_org.shape[0],32*32*3)
X_flat_test_org = X_test_org.reshape(X_test_org.shape[0],32*32*3)

print('Original Sizes:', X_train_org.shape, X_validate_org.shape, Y_train_org.shape, Y_validate_org.shape,X_test_org.shape, Y_test_org.shape)
print('Flattened:', X_flat_train_org.shape, X_flat_validate_org.shape, X_flat_test_org.shape)

In [ ]:
aug_model = joblib.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\\CA1\\with_augmentation\\augmented_Bestmodel.sav")

In [ ]:
print("augmented model train set score: %.2f" % aug_model.score(X_flat_train,Y_train))
print("augmented model validate set score: %.2f" % aug_model.score(X_flat_validate,Y_validate))
print("augmented model test set score: %.2f" % aug_model.score(X_flat_test,Y_test))

In [ ]:
best_model_load = joblib.load("D:\\NUS_ACADEMY\\00computer_intelligence\\CAs\CA1\\without_augmentation\\Bestmodel_MLBP.sav")
print("original model train set score: %.2f" % best_model_load.score(X_flat_train_org,Y_train_org))
print("original model validate set score: %.2f" % best_model_load.score(X_flat_validate_org,Y_validate_org))
print("original model test set score: %.2f" % best_model_load.score(X_flat_test,Y_test))